# FedAvg algorithm with Site-Specific Configurations

In this example, we will demonstrate the FegAvg algorithm using CIFAR10 dataset with site-specific configurations and deploy_map to show how to deploy different "app" to different sites using different configurations. 

We build on top of the previous example [FedAvg algorithm](../sag/sag.ipynb#title)

Please follow these steps before proceeding to the next section:
  * [Understanding FedAvg and SAG](../sag/sag.ipynb#sag)
  * [Convert training code to federated learning training code](../sag/sag.ipynb#code)
  * [Prepare Data](../sag/sag.ipynb#data)


## Job Folder and Configurations

Different sites can have different location for data path, different number of epochs, etc.
We can use Job API to generate the job folder.
Let's first copy the required files over:

In [ ]:
! cp ../code/fl/train.py train.py
! cp ../code/fl/net.py net.py

Then we can run the following cell:

In [ ]:
from net import Net

from nvflare.app_opt.pt.job_config.fed_avg import FedAvgJob
from nvflare.job_config.script_runner import ScriptRunner

if __name__ == "__main__":
    n_clients = 2
    num_rounds = 5
    train_script = "train.py"

    job = FedAvgJob(
        name="cifar10_fedavg_site_specific",
        n_clients=n_clients,
        num_rounds=num_rounds,
        initial_model=Net()
    )

    # Add client "site-1"
    runner = ScriptRunner(
        script=train_script, script_args="--batch_size 4"
    )
    job.to(runner, "site-1")
    
    # Add client "site-2"
    runner = ScriptRunner(
        script=train_script, script_args="--batch_size 6"
    )
    job.to(runner, "site-2")

    job.export_job("/tmp/nvflare/jobs")
    job.simulator_run("/tmp/nvflare/jobs/workdir", gpu="0")


## Run Job

The previous cell exports the job config and executes the job in NVFlare simulator.

If you want to run in production system, you will need to submit this exported job folder to nvflare system.

We can check the content of a job folder using tree command or ls -all


In [ ]:
! tree /tmp/nvflare/jobs/cifar10_fedavg_site_specific